In [71]:
import pandas as pd
import json

# Load the CSV file
file_path = 'new_medicine.csv'
data = pd.read_csv(file_path)

# Handling empty columns
data.fillna("", inplace=True)

# 1. Manufacturer Table
data['manufacturer_id'] = data['product_manufactured'].astype('category').cat.codes + 1
manufacturer_df = data[['manufacturer_id', 'product_manufactured']].drop_duplicates()

# 2. Medicine Details Table
data['product_id'] = data['product_name'].astype('category').cat.codes + 1
medicine_details_df = data[['product_id', 'medicine_desc', 'side_effects']].drop_duplicates()

# 3. Basic Product Info Table
data['sub_category_id'] = data['sub_category'].astype('category').cat.codes + 1
basic_product_info_df = data[['product_id', 'sub_category_id', 'manufacturer_id', 
                                       'product_name', 'product_price']].drop_duplicates()

# 4. Subcategory Table
subcategory_df = data[['sub_category_id', 'sub_category']].drop_duplicates()

# 6. Salt Composition Table
data['salt_id'] = data['salt_composition'].astype('category').cat.codes + 1
salt_composition_df = data[['salt_id', 'salt_composition']].drop_duplicates()

# 5. Product Salt Composition Table
# Assuming 'salt_composition' contains multiple values and needs to be split
product_salt_composition_df = data[['product_id', 'salt_id']].drop_duplicates()

# 8. Drug Interactions Table
interaction_list = []
for idx, row in data.iterrows():
    try:
        interactions = json.loads(row['drug_interactions'])
        for interaction in interactions['drug']:
            interaction_list.append({'product_id': row['product_id'], 'interaction': interaction})
    except Exception as e:
        print(f"Error in row {idx}: {e}")

drug_interactions_df = pd.DataFrame(interaction_list)
drug_interactions_df['interaction_id'] = drug_interactions_df['interaction'].astype('category').cat.codes + 1
drug_interactions_df = drug_interactions_df.drop_duplicates()

# 7. Product Drug Interactions Table
# Merging product_id with interaction_id into a DataFrame
product_drug_interactions_df = pd.DataFrame({
    'product_id': data['product_id'].astype(int),
    'interaction_id': drug_interactions_df['interaction_id']
}).dropna()
product_drug_interactions_df['product_id'] = product_drug_interactions_df['product_id'].astype(int)


# Define the dictionaries
table_dict = {
    'Manufacturer Table': manufacturer_df,
    'Sub Category': subcategory_df,
    'Medicine Details': medicine_details_df,
    'Basic Product Info': basic_product_info_df,
    'Salt Composition': salt_composition_df,
    'Drug Interactions Table': drug_interactions_df,
    'Product Salt Composition': product_salt_composition_df,
    'Product Drug Interactions': product_drug_interactions_df,
}

print(table_dict)
print("Data normalization complete.")

{'Manufacturer Table':         manufacturer_id                 product_manufactured
0                   498           Novo Nordisk India Pvt Ltd
1                   674    Sun Pharmaceutical Industries Ltd
2                   224  Eli Lilly and Company India Pvt Ltd
3                   113                               Biocon
8                   405                            Lupin Ltd
...                 ...                                  ...
176077              310                    Holy Lifesciences
176153              218      Eisen Pharmaceutical Co Pvt Ltd
176158              436        Mediart Life Sciences Pvt Ltd
176161              775                   Zen Medica Pvt Ltd
176162              403                  Lotus Life Sciences

[789 rows x 2 columns], 'Sub Category':         sub_category_id                                    sub_category
0                   117                             Human Insulin Basal
26                  118                            Human Ins

In [72]:
manufacturer_df

,manufacturer_id,product_manufactured
0,498,Novo Nordisk India Pvt Ltd
1,674,Sun Pharmaceutical Industries Ltd
2,224,Eli Lilly and Company India Pvt Ltd
3,113,Biocon
8,405,Lupin Ltd
...,...,...
176077,310,Holy Lifesciences
176153,218,Eisen Pharmaceutical Co Pvt Ltd
176158,436,Mediart Life Sciences Pvt Ltd
176161,775,Zen Medica Pvt Ltd


In [73]:
subcategory_df

,sub_category_id,sub_category
0,117,Human Insulin Basal
26,118,Human Insulin Premix
27,119,Human Insulin Rapid
61,120,Human Insulins And Analogues
141,125,Insulin Analogues Basal
...,...,...
175827,179,Other Mineral Supplements
175837,180,Other Nutrients
176053,185,Other Vitamins
176153,194,Potassium Products


In [74]:
medicine_details_df

,product_id,medicine_desc,side_effects
0,2451,Human Insulatard 40IU/ml Suspension for Inject...,"Hypoglycemia (low blood glucose level),Injecti..."
1,2571,Insulin 40IU/ml Injection is used to improve b...,"Hypoglycemia (low blood glucose level),Injecti..."
2,2470,Huminsulin N 40IU/ml Injection is used to impr...,"Hypoglycemia (low blood glucose level),Injecti..."
3,2562,Insugen-N 40IU/ml Injection is used to improve...,"Hypoglycemia (low blood glucose level),Injecti..."
4,2566,Insulatard 100IU/ml Flexpen is used to improve...,"Hypoglycemia (low blood glucose level),Injecti..."
...,...,...,...
176163,2681,K-Pot 150mg Injection is a potassium salt that...,"Injection site reactions (pain, swelling, redn..."
176164,5557,Troykcl 1.5gm Injection is a potassium salt th...,"Injection site reactions (pain, swelling, redn..."
176165,4655,Ricetral 0.3gm Injection is a potassium salt t...,"Injection site reactions (pain, swelling, redn..."
176166,5569,Tuberium 150mg Injection is a potassium salt t...,"Injection site reactions (pain, swelling, redn..."


In [75]:
basic_product_info_df

,product_id,sub_category_id,manufacturer_id,product_name,product_price
0,2451,117,498,Human Insulatard 40IU/ml Suspension for Injection,₹133.93
1,2571,117,674,Insulin 40IU/ml Injection,₹121.91
2,2470,117,224,Huminsulin N 40IU/ml Injection,₹133.45
3,2562,117,113,Insugen-N 40IU/ml Injection,₹133.36
4,2566,117,498,Insulatard 100IU/ml Flexpen,₹401.03
...,...,...,...,...,...
176163,2681,194,148,K-Pot 150mg Injection,₹9.77
176164,5557,194,718,Troykcl 1.5gm Injection,₹85
176165,4655,194,241,Ricetral 0.3gm Injection,₹4.25
176166,5569,194,578,Tuberium 150mg Injection,₹25.5


In [76]:
salt_composition_df

,salt_id,salt_composition
0,1085,['Insulin Isophane (40IU)']
4,1084,['Insulin Isophane (100IU/ml)']
6,1083,['Insulin Isophane (100IU)']
26,973,['HUMAN PREMIX (100IU)']
27,1017,['Human insulin (40IU)']
...,...,...
176159,1720,['Potassium Chloride (15% w/v)']
176160,1721,['Potassium Chloride (150mg)']
176161,1723,['Potassium Chloride (500mg/5ml)']
176165,1718,['Potassium Chloride (0.3gm)']


In [77]:
drug_interactions_df

,product_id,interaction,interaction_id
0,2451,Benazepril,38
1,2451,Captopril,54
2,2451,Enalapril,97
3,2451,Fosinopril,115
4,2571,Benazepril,38
...,...,...,...
251694,5557,Penicillamine,181
251695,4655,Leflunomide,143
251696,4655,Penicillamine,181
251697,5569,Leflunomide,143


In [78]:
product_salt_composition_df

,product_id,salt_id
0,2451,1085
1,2571,1085
2,2470,1085
3,2562,1085
4,2566,1084
...,...,...
176163,2681,1721
176164,5557,1719
176165,4655,1718
176166,5569,1721


In [79]:
product_drug_interactions_df

,product_id,interaction_id
0,2451,38.0
1,2571,54.0
2,2470,97.0
3,2562,115.0
4,2566,38.0
...,...,...
167366,473,55.0
167367,5225,13.0
167368,204,19.0
167369,1279,25.0


In [81]:
import sqlite3
import pandas as pd

# Define the dictionaries (you need to have these DataFrames ready in your environment)
# manufacturer_df, subcategory_df, etc. should be defined before this

# Export to SQLite Database (.db file)
conn = sqlite3.connect('new_medicine_full.db')
for table_name, df in table_dict.items():
    df.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()

# Export to SQL Dump (.sql file)
conn = sqlite3.connect('new_medicine_full.db')
with open('new_medicine_full.sql', 'w', encoding='utf-8') as f:  # Specify UTF-8 encoding here
    for line in conn.iterdump():
        f.write('%s\n' % line)
conn.close()


In [83]:
import psycopg2

# Database connection parameters
db_name = "medicine_database_work"
db_user = "postgres"
db_password = "123456"
db_host = "localhost"
db_port = 5432

# Establishing the connection
conn = psycopg2.connect(
    database=db_name, 
    user=db_user, 
    password=db_password, 
    host=db_host, 
    port=db_port
)

# Creating a cursor object using the cursor() method
cursor = conn.cursor()

# Reading SQL file with UTF-8 encoding
file_path = 'new_medicine_full.sql'
with open(file_path, 'r', encoding='utf-8') as file:  # Specify UTF-8 encoding here
    sql_script = file.read()

# Executing the script
cursor.execute(sql_script)

# Committing the transaction
conn.commit()

# Closing the connection
cursor.close()
conn.close()
